In [1]:
# import
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
#2020년 기준으로 진행하였다. 이후 data만 바꿔서 2021년, 2022년 진행.

data = pd.read_csv("/Users/janghyun/Desktop/BDP/data/대여이력 정보/data/data2020.csv", index_col=0)




/var/folders/qy/_42009_n085fc984bjdc5v580000gn/T/ipykernel_1825/1931876371.py:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/Users/janghyun/Desktop/BDP/data/대여이력 정보/data/data2020.csv", index_col=0)


In [5]:
#필요없는 column들은 삭제
data.drop(['대여소번호','대여거치대','반납대여소번호','반납거치대'],axis=1,inplace=True)


이상치 삭제 (임의로 작성)

In [6]:
#이용시간이 0인 데이터들은 삭제.
drop_idx = data[data['이용시간']==0].index
data.drop(drop_idx,inplace = True)


In [7]:
#대여소명과 반납소명이 같으면서 이용시간이 30분 미만인 데이터들은 모두 이용거리가 0이었다 -> 삭제
error_data = data[(data['대여대여소명']==data['반납대여소명'])&(data['이용시간']<20)].index
data.drop(error_data,inplace=True)

In [8]:
#이용시간이 10분 이하이면서 이용거리가 5000m 넘는 것들은 이상치라고 판단하고 삭제한다.
error_data2 = data[(data['이용시간']<10)&(data['이용거리']>5000)].sort_values('이용거리').index
data.drop(error_data2,inplace=True)

In [9]:
data.isnull().sum()


자전거번호      0
대여일시       0
대여대여소명     0
반납일시       0
반납대여소명     0
이용시간       0
이용거리      84
dtype: int64

In [11]:
# 이용거리가 0인 것들 다 NaN으로 바꿔준다.

data['이용거리'] = data['이용거리'].replace(0,np.NaN)

In [12]:
data.isnull().sum()

자전거번호            0
대여일시             0
대여대여소명           0
반납일시             0
반납대여소명           0
이용시간             0
이용거리      13517597
dtype: int64

In [13]:
data

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
0,SPB-19866,2020.5.1 0:01,기업은행 앞,2020.5.1 0:03,방학사거리 (봄마당 앞),1.0,330.0
3,SPB-13624,2020.5.1 0:01,영동3교 북단(우성캐릭터 앞 보도),2020.5.1 0:04,3호선 매봉역 3번출구앞,2.0,NaN
4,SPB-32740,2020.5.1 0:00,녹번역 4번출구,2020.5.1 0:05,신한은행 응암동 지점 앞,5.0,NaN
5,SPB-32384,2020.5.1 0:00,잠원역 3번-4번 출구사이,2020.5.1 0:05,한신16차아파트 119동 앞,5.0,NaN
6,SPB-33241,2020.5.1 0:00,대모산입구역 2번출구 앞,2020.5.1 0:05,대청역 1번출구 뒤,5.0,NaN
...,...,...,...,...,...,...,...
20866025,SPB-13688,2020.1.31 23:45,증산3교 앞,2020.1.31 23:59,증산교 앞,13.0,NaN
20866026,SPB-22994,2020.1.31 23:35,명물길 원형무대 앞,2020.1.31 23:59,홍은동 정원여중 입구,21.0,3670.0
20866027,SPB-08546,2020.1.31 23:41,보라매 두산위브 건너편,2020.1.31 23:59,봉천역 2번출구,17.0,3140.0
20866028,SPB-13247,2020.1.31 23:52,수서역 5번출구 뒤,2020.1.31 23:59,문정2동 주민센터,6.0,860.0


결측치 채우기
(이용시간이 0인 데이터들)

결측치를 제외한 값들로 학습시킨 뒤, 선형 회귀 모형을 사용하여 값을 추측하고 결측치를 채워넣는다.

In [23]:
! pip3 install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 6.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.5 MB/s eta 0:00:00


In [24]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [25]:
lin_reg = linear_model.LinearRegression()

In [26]:
data

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
0,SPB-19866,2020.5.1 0:01,기업은행 앞,2020.5.1 0:03,방학사거리 (봄마당 앞),1.0,330.0
3,SPB-13624,2020.5.1 0:01,영동3교 북단(우성캐릭터 앞 보도),2020.5.1 0:04,3호선 매봉역 3번출구앞,2.0,NaN
4,SPB-32740,2020.5.1 0:00,녹번역 4번출구,2020.5.1 0:05,신한은행 응암동 지점 앞,5.0,NaN
5,SPB-32384,2020.5.1 0:00,잠원역 3번-4번 출구사이,2020.5.1 0:05,한신16차아파트 119동 앞,5.0,NaN
6,SPB-33241,2020.5.1 0:00,대모산입구역 2번출구 앞,2020.5.1 0:05,대청역 1번출구 뒤,5.0,NaN
...,...,...,...,...,...,...,...
20866025,SPB-13688,2020.1.31 23:45,증산3교 앞,2020.1.31 23:59,증산교 앞,13.0,NaN
20866026,SPB-22994,2020.1.31 23:35,명물길 원형무대 앞,2020.1.31 23:59,홍은동 정원여중 입구,21.0,3670.0
20866027,SPB-08546,2020.1.31 23:41,보라매 두산위브 건너편,2020.1.31 23:59,봉천역 2번출구,17.0,3140.0
20866028,SPB-13247,2020.1.31 23:52,수서역 5번출구 뒤,2020.1.31 23:59,문정2동 주민센터,6.0,860.0


In [27]:
#데이터에서 이용거리가 NaN인 데이터들을 삭제한다.
df = data.dropna(axis=0)

In [28]:
df

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
0,SPB-19866,2020.5.1 0:01,기업은행 앞,2020.5.1 0:03,방학사거리 (봄마당 앞),1.0,330.0
12,SPB-19925,2020.5.1 0:01,제기역1번출구,2020.5.1 0:06,시립동부병원 앞 사거리,5.0,530.0
15,SPB-15500,2020.5.1 0:00,삼성역 8번출구,2020.5.1 0:07,삼성도로공원,6.0,1250.0
16,SPB-17444,2020.5.1 0:01,제일강산수산입구,2020.5.1 0:07,이디야 커피 수유역점 앞,5.0,940.0
18,SPB-14887,2020.5.1 0:00,교육개발원입구 교차로,2020.5.1 0:07,"국립국악중,고교 정문 맞은편",6.0,1990.0
...,...,...,...,...,...,...,...
20866024,SPB-19370,2020.1.31 23:36,서울광장 옆,2020.1.31 23:59,청운초교 앞 삼거리,21.0,2590.0
20866026,SPB-22994,2020.1.31 23:35,명물길 원형무대 앞,2020.1.31 23:59,홍은동 정원여중 입구,21.0,3670.0
20866027,SPB-08546,2020.1.31 23:41,보라매 두산위브 건너편,2020.1.31 23:59,봉천역 2번출구,17.0,3140.0
20866028,SPB-13247,2020.1.31 23:52,수서역 5번출구 뒤,2020.1.31 23:59,문정2동 주민센터,6.0,860.0


In [29]:
#그 데이터들로 train, test set을 나눈다.

X = df[['이용시간']]
y = df[['이용거리']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=1004)

In [30]:
print(X_train.shape)
print(X_test.shape)


(5469132, 1)
(1367283, 1)


In [31]:
lin_reg_model = lin_reg.fit(X_train,y_train)

In [32]:
y_pred = lin_reg_model.predict(X_test)
print(r2_score(y_test,y_pred))

#결정계수가 대략 0.40 정도 나온다.

0.400349577552116


In [33]:
# data['이용시간']을 모델에 넣고 all_pred에 담는다.

all_pred = lin_reg_model.predict(data.loc[:,['이용시간']])
all_pred

array([[ 898.03920176],
       [1016.48684746],
       [1371.82978458],
       ...,
       [2793.20153303],
       [1490.27743028],
       [1134.93449317]])

In [34]:
# null 값이 발견되면 그 자리에 예측값으로 대체한다.

data['이용거리']= np.where(data['이용거리'].isnull(),
                       pd.Series(all_pred.flatten()),
                       data['이용거리'])

In [35]:
data.isnull().sum()

자전거번호     0
대여일시      0
대여대여소명    0
반납일시      0
반납대여소명    0
이용시간      0
이용거리      0
dtype: int64

In [36]:
data[data['이용시간']==2]

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
3,SPB-13624,2020.5.1 0:01,영동3교 북단(우성캐릭터 앞 보도),2020.5.1 0:04,3호선 매봉역 3번출구앞,2.0,1016.486847
9,SPB-32335,2020.5.1 0:04,CJ 드림시티,2020.5.1 0:06,이마트 은평점,2.0,1016.486847
82,SPB-17842,2020.5.1 0:09,동방1교,2020.5.1 0:12,도림천 신화교,2.0,600.000000
177,SPB-20055,2020.5.1 0:14,망우역 1번출구,2020.5.1 0:17,상봉터미널2,2.0,230.000000
196,SPB-03816,2020.5.1 0:15,서강대 후문 옆,2020.5.1 0:18,대흥역 1번출구,2.0,440.000000
...,...,...,...,...,...,...,...
20865807,SPB-13627,2020.1.31 23:45,양평동6차현대아파트 앞,2020.1.31 23:48,양평1 보행육교 앞,2.0,600.000000
20865839,SPB-23714,2020.1.31 23:47,월곡역 3번출구,2020.1.31 23:49,생명의 전화 종합복지관 앞 교차로,2.0,290.000000
20865848,SPB-23893,2020.1.31 23:46,중화역 2번출구,2020.1.31 23:49,장안중학교,2.0,260.000000
20865958,SPB-19329,2020.1.31 23:53,중랑전화국 교차로,2020.1.31 23:56,하나은행 면목지점,2.0,570.000000


In [37]:
data.drop(['대여대여소명','반납대여소명','반납일시'],axis=1,inplace=True)

In [38]:
data

,자전거번호,대여일시,이용시간,이용거리
0,SPB-19866,2020.5.1 0:01,1.0,330.000000
3,SPB-13624,2020.5.1 0:01,2.0,1016.486847
4,SPB-32740,2020.5.1 0:00,5.0,1371.829785
5,SPB-32384,2020.5.1 0:00,5.0,1371.829785
6,SPB-33241,2020.5.1 0:00,5.0,1371.829785
...,...,...,...,...
20866025,SPB-13688,2020.1.31 23:45,13.0,2319.410950
20866026,SPB-22994,2020.1.31 23:35,21.0,3670.000000
20866027,SPB-08546,2020.1.31 23:41,17.0,3140.000000
20866028,SPB-13247,2020.1.31 23:52,6.0,860.000000


In [39]:
data["대여일시"]=pd.to_datetime(data['대여일시'].str.strip(), format="%Y-%m-%d %H:%M", errors='coerce')

In [40]:
data.sort_values("대여일시")

,자전거번호,대여일시,이용시간,이용거리
20100750,SPB-16682,2020-01-01 00:00:00,9.0,1760.000000
20100806,SPB-10534,2020-01-01 00:00:00,20.0,3200.000000
20100783,SPB-13778,2020-01-01 00:01:00,15.0,2556.306242
20100754,SPB-08263,2020-01-01 00:01:00,10.0,1720.000000
20102253,SPB-18109,2020-01-01 00:01:00,142.0,250.000000
...,...,...,...,...
16220747,SPB-51158,2020-12-31 23:59:06,8.0,1727.172722
16220736,SPB-37040,2020-12-31 23:59:20,4.0,667.170000
16220822,SPB-50200,2020-12-31 23:59:45,35.0,4925.259156
16220800,SPB-30203,2020-12-31 23:59:49,23.0,3981.810000


In [41]:
data.isnull().sum()

자전거번호    0
대여일시     0
이용시간     0
이용거리     0
dtype: int64

In [42]:
data["year"] = data["대여일시"].dt.year
data["month"] = data["대여일시"].dt.month

In [45]:
data.drop("대여일시", axis = 1, inplace=True)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20354012 entries, 0 to 20866029
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   자전거번호   object 
 1   이용시간    float64
 2   이용거리    float64
 3   year    int64  
 4   month   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1.4+ GB


In [49]:
data.isnull().sum()

자전거번호    0
이용시간     0
이용거리     0
year     0
month    0
dtype: int64

In [51]:
data

,자전거번호,이용시간,이용거리,year,month
0,SPB-19866,1.0,330.000000,2020,5
3,SPB-13624,2.0,1016.486847,2020,5
4,SPB-32740,5.0,1371.829785,2020,5
5,SPB-32384,5.0,1371.829785,2020,5
6,SPB-33241,5.0,1371.829785,2020,5
...,...,...,...,...,...
20866025,SPB-13688,13.0,2319.410950,2020,1
20866026,SPB-22994,21.0,3670.000000,2020,1
20866027,SPB-08546,17.0,3140.000000,2020,1
20866028,SPB-13247,6.0,860.000000,2020,1


In [50]:
data.to_csv("data2020_pre.csv")
